In [ ]:
!pip install pytorch-tabnet

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.applications import EfficientNetB0
from transformers import TFBertModel, BertTokenizer
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
import re
from src.utils import download_images

In [ ]:

# Load datasets
train_df = pd.read_csv('/content/train_subset1.csv')
test_df = pd.read_csv('/content/test_subset1.csv')

from src.utils import download_images
import src.constants
import pandas as pd

# Load the train dataset
train_df = pd.read_csv('/content/train_subset1.csv')

# Download images
download_images(train_df['image_link'].tolist(), 'images/')

In [ ]:

# Image preprocessing function
def load_and_preprocess_image(img_name, img_dir):
    img_path = f"{img_dir}/{img_name}"
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize the image
    return img


In [ ]:

# Load pre-trained EfficientNet model (without top layers)
base_model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

In [ ]:

# Function to extract numeric value and unit from entity_value
def extract_numeric_and_unit(value):
    match = re.match(r"([\d\.]+)\s*(\w+)", value)
    if match:
        numeric_part = float(match.group(1))
        unit_part = match.group(2).lower()
        return numeric_part, unit_part
    return None, None


In [ ]:

# Preprocess and extract features for train data
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True # Add this line to handle truncated images

train_df['image_name'] = train_df['image_link'].apply(lambda url: url.split('/')[-1])
train_image_features = np.array([base_model.predict(load_and_preprocess_image(img_name, '/content/images'))
                                 for img_name in train_df['image_name']])
train_image_features = train_image_features.reshape(len(train_df), -1)

# Apply numeric value and unit extraction to train dataset
train_df['numeric_value'], train_df['unit'] = zip(*train_df['entity_value'].map(extract_numeric_and_unit))
train_df['combined_text'] = train_df['entity_name'] + ' ' + train_df['group_id'].astype(str) + ' ' + train_df['unit']


In [ ]:
train_df.isna().sum()

In [ ]:
train_df.fillna({'numeric_value': 0, 'unit': 'unknown','combined_text': 'unknown'}, inplace=True)


In [ ]:

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Function to encode text using BERT
def encode_text(texts):
    encoded_inputs = tokenizer(texts.tolist(), padding=True, truncation=True, return_tensors='tf')
    outputs = bert_model(encoded_inputs).last_hidden_state[:, 0, :]
    return outputs

# Encode the combined text using BERT for train data
train_text_embeddings = encode_text(train_df['combined_text'])
combined_train_features = np.concatenate([train_image_features, train_text_embeddings.numpy()], axis=1)

In [ ]:
combined_train_features

In [ ]:
# Train-test split
X_train, X_val, y_train, y_val = train_test_split(combined_train_features, train_df['numeric_value'], test_size=0.2, random_state=42)



In [ ]:
y_train = y_train.values.reshape(-1, 1)
y_val = y_val.values.reshape(-1, 1)

In [ ]:


# Initialize and train TabNet model
tabnet_model = TabNetRegressor()
tabnet_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_name=['val'],
    eval_metric=['rmse'],
    max_epochs=50,
    patience=5,
    batch_size=32,
    virtual_batch_size=16
)


In [ ]:
y_test_pred = tabnet_model.predict(X_val)

In [ ]:
from sklearn.metrics import f1_score

# Define a threshold to classify predictions (you can choose a different method)
threshold = train_df['numeric_value'].median()

# Convert actual values to binary classes based on the threshold
y_true = (y_val >= threshold).astype(int)

# Convert predictions to binary classes based on the same threshold
y_pred = (y_test_pred.flatten() >= threshold).astype(int)

# Calculate F1 score
f1 = f1_score(y_true, y_pred)

print("F1 Score:", f1)


In [ ]:
from src.utils import download_images
import src.constants
import pandas as pd

# Load the train dataset


# Download images
download_images(test_df['image_link'].tolist(), 'images2/')

In [ ]:





# Preprocess and extract features for test data
test_df['image_name'] = test_df['image_link'].apply(lambda url: url.split('/')[-1])
test_image_features = np.array([base_model.predict(load_and_preprocess_image(img_name, '/content/images2'))
                                for img_name in test_df['image_name']])
test_image_features = test_image_features.reshape(len(test_df), -1)

# Combine text features for the test data
test_df['combined_text'] = test_df['entity_name'] + ' ' + test_df['group_id'].astype(str)
test_text_embeddings = encode_text(test_df['combined_text'])

# Combine image and text features for the test data
combined_test_features = np.concatenate([test_image_features, test_text_embeddings.numpy()], axis=1)
# Predict the numeric values for the test data
y_test_pred = tabnet_model.predict(combined_test_features)

# Ensure y_test_pred is a 1D array
y_test_pred_flat = y_test_pred.flatten()

# Combine predictions with units for the final output
test_df['predicted_entity_value'] = [f"{pred:.2f}" for pred, unit in zip(y_test_pred_flat, test_df['unit'])]


# Prepare the final output in the required format
test_output = test_df[['index', 'predicted_entity_value']]

# Save the output to a CSV file
test_output.to_csv('sample_test_out2.csv', index=False)
